# Capstone final project

In [1]:
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


## Introduction/Business Problem

Denver Colorado Is a very busy place full of grocery stores, resturants, and other small businesses. Part of this is due to the masssive inflow of people moving to Colorado for business and other factors. Since there is a lot of people moving into a place they have never been before I wanted to create a module that will help them determine the best places to live. The plan is to look at three diffrent datasets that provide diffrent information one may consider when moving to a place. These are things like cost of housing, crime in the area, and the proximity of things like grocery stores and other food related shops.

## Data used

Housing data - https://www.denvergov.org/opendata/dataset/city-and-county-of-denver-affordable-housing-units

Food data - Couldn't aquire the link 

Crime data - https://www.denvergov.org/opendata/dataset/city-and-county-of-denver-crime